# Exercise 2_5 Digital Image Processing

Amirkabir University of Technology

Dr. Rahmati

by Gholamreza Dar

Spring 2022

## Imports

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns; sns.set()
import cv2
from numba import jit, njit
import datetime

sns.set_style("dark")

import logging;
logger = logging.getLogger("numba");
logger.setLevel(logging.ERROR)


## Functions

### Helpers

In [ ]:
def rgb(img):
    return cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

def bgr(img):
    return cv2.cvtColor(img, cv2.COLOR_RGB2BGR)

def gray(img):
    return cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)

In [ ]:
def disp(img, title=None, s=10):
    plt.figure(figsize=(s,s))
    if title is not None:
        plt.title(title)
    plt.axis('off')
    plt.imshow(img, cmap='gray')
    plt.show()

### Seam Carving

#### Utility Functions

In [ ]:
def calculate_energy_map(image, mask=None):
    """ Calculate energy map for given image.

    Args:
        image (np.uint8 3 channels): input image
        mask (np.uint8 1 channel, optional): mask. Defaults to None.

    Returns:
        np.float32 1 channel: energy map
    """
    image_gray = gray(image)
    delta_x = cv2.Sobel(image_gray, cv2.CV_64F, 0, 1, ksize=5)
    delta_y = cv2.Sobel(image_gray, cv2.CV_64F, 1, 0, ksize=5)
    gradient_magnitude = np.sqrt(delta_x**2 + delta_y**2)
    gradient_magnitude /= gradient_magnitude.max()/255

    if mask is not None:
        gradient_magnitude[mask > 127] = -10000
    
    return gradient_magnitude

In [ ]:
@jit
def find_best_seam(energy_map, debug=False):
    """ Find next seam.
    
    Args:
        energy_map (np.float32 1 channel): energy map
        debug (bool, optional): debug mode. Defaults to False.
        
    Returns:
        np.int32 : cols of next best seam
        int : energy of next best seam
    """

    # 1. Find the minimum energy path matrix(dp)
    U = np.zeros_like(energy_map)

    for i in range(energy_map.shape[0]-1, -1, -1):
        for j in range(energy_map.shape[1]):
            if i==energy_map.shape[0]-1:
                U[i, j] = energy_map[i, j]
            elif j==0:
                U[i, j] = energy_map[i, j] + min(U[i+1, j], U[i+1, j+1])
            elif j==energy_map.shape[1]-1:
                U[i, j] = energy_map[i, j] + min(U[i+1, j-1], U[i+1, j])
            else:
                U[i, j] = energy_map[i, j] + min(U[i+1, j-1], U[i+1, j], U[i+1, j+1])
    
    # 2. Find the minimum energy path
    seam = np.zeros((U.shape[0], ), dtype='uint32')
    last_rows_min_col = np.argmin(U[0])
    seam_energy = energy_map[0, last_rows_min_col]
    seam[0] = last_rows_min_col

    for i in range(1, U.shape[0]):
        if U[i, last_rows_min_col] == U[i-1, last_rows_min_col]:
            last_rows_min_col = last_rows_min_col
        elif last_rows_min_col == 0:
            if U[i, last_rows_min_col]>=U[i, last_rows_min_col+1]:
                last_rows_min_col = last_rows_min_col+1
        elif last_rows_min_col == U.shape[1]-1:
            if U[i, last_rows_min_col]>=U[i, last_rows_min_col-1]:
                last_rows_min_col = last_rows_min_col-1
        else:
            if U[i, last_rows_min_col-1] == U[i, last_rows_min_col] and U[i, last_rows_min_col] == U[i, last_rows_min_col+1]:
                dir = np.random.randint(-1, 2)
                # dir = 0
                last_rows_min_col = last_rows_min_col + dir
            elif U[i, last_rows_min_col]>=U[i, last_rows_min_col-1]:
                if U[i, last_rows_min_col-1] >= U[i, last_rows_min_col+1]:
                    last_rows_min_col = last_rows_min_col+1
                else:
                    last_rows_min_col = last_rows_min_col-1
            else:
                if U[i, last_rows_min_col] >= U[i, last_rows_min_col+1]:
                    last_rows_min_col = last_rows_min_col+1
                else:
                    last_rows_min_col = last_rows_min_col
        
        seam[i] = last_rows_min_col
        seam_energy += energy_map[i, last_rows_min_col]

    # 3. Visualize the seam
    if debug:
        disp(U, "U", s=8)
        # print(U)

    # 4. Return the seam and its energy
    return seam, seam_energy

In [ ]:
def remove_seam(image, seam, mask=None):
    """ Remove the seam from energy map and the mask if it exists.

    Args:
        image (np.uint8 3 channels): image
        seam (np.int32 array): seam to be removed
        mask (np.uint8 1 channel, optional): mask. Defaults to None.

    Returns:
        np.uint8 3 channels: resized image without the "seam"
        np.uint8 1 channel: resized mask without the "seam" if mask exists
    """
    if mask is not None:
        idx=np.ones_like(image, bool)
        idx_mask=np.ones_like(mask, bool)
        for i in range(image.shape[0]):
            idx[i, int(seam[i])]=False
            idx_mask[i, int(seam[i])]=False
        reduced_image = image[idx].reshape(image.shape[0], image.shape[1]-1, 3)
        reduced_mask = mask[idx_mask].reshape(mask.shape[0], mask.shape[1]-1)
        return reduced_image, reduced_mask
    else:
        idx=np.ones_like(image, bool)
        for i in range(image.shape[0]):
            idx[i, int(seam[i])]=False
        reduced_image = image[idx].reshape(image.shape[0], image.shape[1]-1, 3)
        return reduced_image

In [ ]:
@jit
def insert_seam(image, seam):
    """ Insert the seam into the image by averaging the seams to its left and right.

    Args:
        image (np.uint8 3 channels): image(RGB)
        seam (np.int32 array): seam to be inserted
    
    Returns:
        np.uint8 3 channels: resized image with the new "seam" inserted. width = image_width+1
    """

    # New Image is one column wider
    resized_image = np.zeros((image.shape[0], image.shape[1]+1, 3), dtype='uint8')

    # For every row
    for i in range(image.shape[0]):
        j = seam[i]

        # Find the value for the pixel on the seam at row i
        new_value = np.array([0, 0, 0])
        if j==0:
            new_value = ((image[i, j].astype(np.float32) + image[i, j+1].astype(np.float32))/2).astype(np.uint8)
        else:
            new_value = ((image[i, j-1].astype(np.float32) + image[i, j].astype(np.float32))/2).astype(np.uint8)
        
        # Insert the new value in the image
        resized_image[i, :j] = image[i, :j] # Pixels to the left of the seam
        resized_image[i, j] = new_value # Pixels on the seam
        resized_image[i, j+1:] = image[i, j:]  # Pixels to the right of the seam
    
    return resized_image

#### Main Functions

In [ ]:
@jit
def shrink_image(image, num_seams, seam_dir="vertical", mask=None, save_animation=False, color=[255,0,0]):
    """ Shrink the image by removing num_seams seams.

    Args:
        image (np.uint8 3 channel): image to be shrinked
        num_seams (int): number of seams to be removed
        mask (np.uint8 1 channel, optional): mask. Defaults to None.

    Returns:
        np.uint8 3 channel: shrinked image
        np.uint8 1 channel: shrinked mask if mask exists
    """
    # Timestamp for saving the animation
    time_stamp = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

    # Rotate the image 90 degrees if seam_dir is horizontal
    if seam_dir=="horizontal":
        image = np.rot90(image, 1)
        if mask is not None:
            mask = np.rot90(mask, 1)

    # Do the seam carving on the image and mask if it exists
    for i in range(num_seams):
        # 1. Find the seam
        if mask is not None:
            energy_map = calculate_energy_map(image, mask)
        else:
            energy_map = calculate_energy_map(image)
        seam, seam_energy = find_best_seam(energy_map)
        
        # 2. Vizualize the seam
        if save_animation:
            drawn_image = draw_seam(image, seam, color=color)
            if seam_dir=="horizontal":
                drawn_image = np.rot90(drawn_image, 3)
            cv2.imwrite(f"P5_result/animation/seam_carving_{time_stamp}_{str(i).zfill(4)}.jpg", bgr(drawn_image))

        # 3. Remove the seam
        if mask is not None:
            image, mask = remove_seam(image, seam, mask)
        else:
            image = remove_seam(image, seam)

    # Revert the Rotation if seam_dir is horizontal
    if seam_dir=="horizontal":
        image = np.rot90(image, 3)
        if mask is not None:
            mask = np.rot90(mask, 3)

    if mask is not None:
        return image, mask
    else:
        return image

In [ ]:
@jit
def expand_image_naive(image, num_seams, seam_dir="vertical", save_animation=False, color=[255,0,0]):
    """ Expand the image by always inserting the best seam.

    Args:
        image (np.uint8 3 channel): image to be expanded
        num_seams (int): number of seams to be inserted
    
    Returns:
        np.uint8 3 channel: expanded image

    """
    # Timestamp for saving the animation
    time_stamp = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

    # Rotate the image 90 degrees if seam_dir is horizontal
    if seam_dir=="horizontal":
        image = np.rot90(image, 1)

    # Do the seam insertion on the image
    for i in range(num_seams):
        # 1. Find the seam
        energy_map = calculate_energy_map(image)
        seam, seam_energy = find_best_seam(energy_map)
        
        # 2. Vizualize the seam
        if save_animation:
            drawn_image = draw_seam(image, seam, color=color)
            if seam_dir=="horizontal":
                drawn_image = np.rot90(drawn_image, 3)
            cv2.imwrite(f"P5_result/animation/seam_insertion_{time_stamp}_{str(i).zfill(4)}.jpg", bgr(drawn_image))

        # 3. Insert the seam
        image = insert_seam(image, seam)

    # Revert the Rotation if seam_dir is horizontal
    if seam_dir=="horizontal":
        image = np.rot90(image, 3)
    
    return image

In [ ]:
@jit
def expand_image(image, num_seams, seam_dir="vertical", save_animation=False, color=[255,0,0]):
    """ Expand the image by always inserting the top 'num_seams' seams.

    Args:
        image (np.uint8 3 channel): image to be expanded
        num_seams (int): number of seams to be inserted
    
    Returns:
        np.uint8 3 channel: expanded image

    """
    # Timestamp for saving the animation
    time_stamp = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

    # Rotate the image 90 degrees if seam_dir is horizontal
    if seam_dir=="horizontal":
        image = np.rot90(image, 1)

    # 1. Find the seams by iteratively removing the top seam
    seams = []
    temp_image = image.copy()
    for i in range(num_seams):
        energy_map = calculate_energy_map(temp_image)
        seam, seam_energy = find_best_seam(energy_map)
        temp_image = remove_seam(temp_image, seam)
        seams.append(seam)

    # 2. Do the seam insertion on the image
    for i in range(num_seams):
        # 1. Get the seam
        seam = seams[i]
        
        # 2. Vizualize the seam
        if save_animation:
            drawn_image = draw_seam(image, seam, color=color)
            if seam_dir=="horizontal":
                drawn_image = np.rot90(drawn_image, 3)
            cv2.imwrite(f"P5_result/animation/seam_insertion_{time_stamp}_{str(i).zfill(4)}.jpg", bgr(drawn_image))

        # 3. Insert the seam
        image = insert_seam(image, seam)

        # 4. Update the seams columns (because they were extracted when the image was shrunk)
        for j in range(i+1, num_seams):
            seams[j][np.where(seams[j] >= seam)] += 2  # "1" because of the inserted seam and "1" because they were extracted when the image was shrunk

    # Revert the Rotation if seam_dir is horizontal
    if seam_dir=="horizontal":
        image = np.rot90(image, 3)
    
    return image

In [ ]:
def remove_object(image, mask, color=[255,0,0], seam_dir="vertical", cmap="Reds", on_image=False, save_animation=False, debug=True):

    # Timestamp for saving the animation
    time_stamp = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

    # Rotate the image 90 degrees if seam_dir is horizontal
    if seam_dir=="horizontal":
        image = np.rot90(image, 1)
        if mask is not None:
            mask = np.rot90(mask, 1)
    
    SEAMS_NEEDED = 0

    # Remove the object from the image
    shrunk_image = image.copy()
    mask_copy = mask.copy()
    while (mask_copy>127).sum() > 0:
        SEAMS_NEEDED += 1
        energy_temp = calculate_energy_map(shrunk_image, mask=mask_copy)
        seam, energy = find_best_seam(energy_temp)
        shrunk_image, mask_copy = remove_seam(shrunk_image, seam, mask=mask_copy)
        
        if save_animation:
            drawn_image = draw_seam(shrunk_image, seam, color=color)
            if seam_dir=="horizontal":
                drawn_image = np.rot90(drawn_image, 3)
            cv2.imwrite(f"P5_result/animation/seam_removal_{time_stamp}_{str(SEAMS_NEEDED).zfill(4)}.jpg", bgr(drawn_image))

    if debug:
        if seam_dir=="horizontal":
            shrunk_image_temp = np.rot90(shrunk_image, 3)
        else:
            shrunk_image_temp = shrunk_image.copy()
        disp(shrunk_image_temp, "Shrunk Image", s=15)
        print("shrunk_image_temp.shape", shrunk_image_temp.shape)
        print(f"Removed {SEAMS_NEEDED} seams to remove the object entirely")

    # Scale back the image to the original size
    expanded_image = expand_image(shrunk_image, SEAMS_NEEDED, seam_dir=seam_dir, save_animation=save_animation, color=color)

    if debug:
        if seam_dir=="horizontal":
            expanded_image_temp = np.rot90(expanded_image, 3)
            image_temp = np.rot90(image, 3)
        else:
            expanded_image_temp = expanded_image.copy()
            image_temp = image.copy()

        disp(expanded_image_temp, title="Expanded Image", s=15)
        disp(image_temp, title="Original", s=15)
        print("image.shape", image.shape)
        print("expanded_image.shape", expanded_image_temp.shape)


    if debug:
        seams_drawn = visualize_best_seams(image, num_seams=SEAMS_NEEDED, mask=mask, seam_dir=seam_dir, cmap=cmap, on_image=on_image)
        if seam_dir=="horizontal":
            seams_drawn = np.rot90(seams_drawn, 3)
        disp(seams_drawn, title="Best Seams", s=15)

    # Revert the Rotation if seam_dir is horizontal
    if seam_dir=="horizontal":
        expanded_image = np.rot90(expanded_image, 3)
        shrunk_image = np.rot90(shrunk_image, 3)

    return expanded_image, shrunk_image

    

#### Visualization Functions 

In [ ]:
def draw_seam(image, seam, color=[255,0,0]):
    """ Draw the seam on the image.
    
    Args:
        image (np.uint8 3 channels): input image
        seam (np.int32 array): cols of the seam
        color (list[3], optional): color of the seam. Defaults to [255,0,0].
    
    """
    image = image.copy()

    for i in range(image.shape[0]):
        if seam[i] < image.shape[1]:
            image[i, seam[i]] = color

    return image

In [ ]:
@jit
def visualize_best_seams(image, num_seams, seam_dir="vertical", mask=None, cmap="summer", on_image=True):

    # Timestamp for saving the animation
    time_stamp = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

    # Rotate the image 90 degrees if seam_dir is horizontal
    if seam_dir=="horizontal":
        image = np.rot90(image, 1)

    # 1. Find the seams by iteratively removing the top seam
    seams = []
    seams_energy = []
    temp_image = image.copy()
    temp_mask = mask.copy() if mask is not None else None
    for i in range(num_seams):
        if mask is not None:
            energy_map = calculate_energy_map(temp_image, temp_mask)
        else:
            energy_map = calculate_energy_map(temp_image)
        seam, seam_energy = find_best_seam(energy_map)

        if mask is not None:
            temp_image, temp_mask = remove_seam(temp_image, seam, temp_mask)
        else:
            temp_image = remove_seam(temp_image, seam)

        seams.append(seam)
        seams_energy.append(seam_energy)

    # 2. Choose the drawing image
    if on_image:
        drawn_image = image.copy()
    else:
        # draw on the energy map
        if mask is not None:
            energy_map = calculate_energy_map(image, mask).astype(np.uint8)
        else:
            energy_map = calculate_energy_map(image).astype(np.uint8)
        
        # Convert energy map from 1 channel to 3 channels
        energy_map_rgb = np.stack([energy_map]*3, axis=-1)
        drawn_image = energy_map_rgb.copy()

    # 3. Draw the seams on the image
    for i in range(num_seams):
        # 1. Get the seam
        seam = seams[i]

        # 2. Draw the seam
        norm = matplotlib.colors.Normalize(vmin=min(seams_energy), vmax=max(seams_energy))
        cmap = matplotlib.cm.get_cmap(cmap)
        color = cmap(norm(seams_energy[i]))
        
        drawn_image = draw_seam(drawn_image, seam, color=[color[0]*255, color[1]*255, color[2]*255])

        # 4. Update the seams columns (because they were extracted when the image was shrunk)
        for j in range(i+1, num_seams):
            seams[j][np.where(seams[j] >= seam)] += 1  # "1" because they were extracted when the image was shrunk

    # Revert the Rotation if seam_dir is horizontal
    if seam_dir=="horizontal":
        drawn_image = np.rot90(drawn_image, 3)
    
    return drawn_image

## Main

In [ ]:
towers = rgb(cv2.imread('inputs/P5/towers.jpg'))
landscape = rgb(cv2.imread('inputs/P5/landscape.jpg'))
muqarnas = rgb(cv2.imread('inputs/P5/islamic.jpg'))
lizzy = rgb(cv2.imread('inputs/P5/lizzys.jpg'))
site = rgb(cv2.imread('inputs/P5/site3s.png'))
trump1 = rgb(cv2.imread('inputs/P5/donald_plays_golf.png'))
trump1_blur = rgb(cv2.imread('inputs/P5/donald_plays_golf_blur.png'))
trump2 = rgb(cv2.imread('inputs/P5/donald_plays_tennis.png'))
trump_putin = rgb(cv2.imread('inputs/P5/trump_putin2.png'))
trump_putin_mask = cv2.imread('inputs/P5/trump_putin_mask.png', 0)
trump_queen = rgb(cv2.imread('inputs/P5/trump_queen6.png'))
trump_queen_mask = cv2.imread('inputs/P5/trump_queen_mask.png', 0)
disp(site)

### Display Energy Map

In [ ]:
disp(calculate_energy_map(trump_putin), title="Energy Map", s=15)

### a) Shrink Horizontally (Trump2)

In [ ]:
# Shrink by 10%
w = trump2.shape[1]

trump2_shrunk = shrink_image(trump2, num_seams=int(w*0.1), seam_dir="vertical")
trump2_viz = visualize_best_seams(trump2, num_seams=int(w*0.1), seam_dir="vertical", on_image=False, cmap="Reds")

disp(trump2_shrunk, title="Shrunk by 10% horizontally", s=15) 
disp(trump2_viz, "Top 10% seams", s=15)

In [ ]:
# Shrink by 25%
w = trump2.shape[1]

trump2_shrunk = shrink_image(trump2, num_seams=int(w*0.25), seam_dir="vertical")
trump2_viz = visualize_best_seams(trump2, num_seams=int(w*0.25), seam_dir="vertical", on_image=False, cmap="Reds")

disp(trump2_shrunk, title="Shrunk by 25% horizontally", s=15) 
disp(trump2_viz, "Top 25% seams", s=15)

In [ ]:
# Shrink by 50%
w = trump2.shape[1]

trump2_shrunk = shrink_image(trump2, num_seams=int(w*0.5), seam_dir="vertical")
trump2_viz = visualize_best_seams(trump2, num_seams=int(w*0.5), seam_dir="vertical", on_image=False, cmap="Reds")

disp(trump2_shrunk, title="Shrunk by 50% horizontally", s=15) 
disp(trump2_viz, "Top 50% seams", s=15)

### b) Shrink Vertically (Trump1)

In [ ]:
# Shrink by 10%
h = trump1.shape[0]

trump1_shrunk = shrink_image(trump1, num_seams=int(h*0.1), seam_dir="horizontal")
trump1_viz = visualize_best_seams(trump1, num_seams=int(h*0.1), seam_dir="horizontal", on_image=False, cmap="summer")

disp(trump1_shrunk, title="Shrunk by 10% vertically", s=15) 
disp(trump1_viz, "Top 10% seams", s=15)

In [ ]:
# Shrink by 25%
h = trump1.shape[0]

trump1_shrunk = shrink_image(trump1, num_seams=int(h*0.25), seam_dir="horizontal")
trump1_viz = visualize_best_seams(trump1, num_seams=int(h*0.25), seam_dir="horizontal", on_image=False, cmap="summer")

disp(trump1_shrunk, title="Shrunk by 25% vertically", s=15) 
disp(trump1_viz, "Top 25% seams", s=15)

In [ ]:
# Shrink by 50%
h = trump1.shape[0]

trump1_shrunk = shrink_image(trump1, num_seams=int(h*0.5), seam_dir="horizontal")
trump1_viz = visualize_best_seams(trump1, num_seams=int(h*0.5), seam_dir="horizontal", on_image=False, cmap="summer")

disp(trump1_shrunk, title="Shrunk by 50% vertically", s=15) 
disp(trump1_viz, "Top 50% seams", s=15)

#### Blur the grass

In [ ]:
# Shrink by 50%
h = trump1_blur.shape[0]

trump1_blur_shrunk = shrink_image(trump1_blur, num_seams=int(h*0.5), seam_dir="horizontal")
trump1_blur_viz = visualize_best_seams(trump1_blur, num_seams=int(h*0.5), seam_dir="horizontal", on_image=False, cmap="summer")

disp(trump1_blur_shrunk, title="Shrunk by 50% vertically", s=15) 
disp(trump1_blur_viz, "Top 50% seams", s=15)

### c) Shrink Horizontally Landscape

In [ ]:
# Shrink by 30%
w = landscape.shape[1]

landscape_shrunk = shrink_image(landscape, num_seams=int(w*0.3), seam_dir="vertical")
landscape_viz = visualize_best_seams(landscape, num_seams=int(w*0.3), seam_dir="vertical", on_image=False, cmap="Reds")

disp(landscape_shrunk, title="Shrunk by 30% horizontally", s=15) 
disp(landscape_viz, "Top 30% seams", s=15)

In [ ]:
# Shrink by 30%
h = landscape.shape[0]

landscape_shrunk = shrink_image(landscape, num_seams=int(h*0.3), seam_dir="horizontal")
landscape_viz = visualize_best_seams(landscape, num_seams=int(h*0.3), seam_dir="horizontal", on_image=False, cmap="summer")

disp(landscape_shrunk, title="Shrunk by 30% vertically", s=15) 
disp(landscape_viz, "Top 30% seams", s=15)

### Site

In [ ]:
# Shrink by 20%
w = site.shape[1]

site_shrunk = shrink_image(site, num_seams=int(w*0.20), seam_dir="vertical")
site_viz = visualize_best_seams(site, num_seams=int(w*0.20), seam_dir="vertical", on_image=False, cmap="Reds")

disp(site_shrunk, title="Shrunk by 20% horizontally", s=15) 
disp(site_viz, "Top 20% seams", s=15)


h = site.shape[0]

site_shrunk2 = shrink_image(site_shrunk, num_seams=int(h*0.40), seam_dir="horizontal")
site_viz2 = visualize_best_seams(site_shrunk, num_seams=int(h*0.40), seam_dir="horizontal", on_image=False, cmap="Reds")

disp(site_shrunk2, title="Shrunk by 20% vertically", s=15) 
disp(site_viz2, "Top 20% seams", s=15)


### d) A Bad example

In [ ]:
# Shrink by 50%
w = muqarnas.shape[1]

muqarnas_shrunk = shrink_image(muqarnas, num_seams=int(w*0.5), seam_dir="vertical")
muqarnas_viz = visualize_best_seams(muqarnas, num_seams=int(w*0.5), seam_dir="vertical", on_image=False, cmap="Reds")

disp(muqarnas_shrunk, title="Shrunk by 50% horizontally", s=15) 
disp(muqarnas_viz, "Top 50% seams", s=15)

In [ ]:
# Shrink by 30%
w = lizzy.shape[1]

lizzy_shrunk = shrink_image(lizzy, num_seams=int(w*0.3), seam_dir="vertical")
lizzy_viz = visualize_best_seams(lizzy, num_seams=int(w*0.3), seam_dir="vertical", on_image=False, cmap="Reds")

disp(lizzy_shrunk, title="Shrunk by 30% horizontally", s=15) 
disp(lizzy_viz, "Top 30% seams", s=15)

### e) Expand images

In [ ]:
image = trump2
w = image.shape[1]
expanded_image = expand_image(image, num_seams=int(w*0.1), seam_dir="vertical")
viz = visualize_best_seams(image, num_seams=int(w*0.1), seam_dir="vertical", on_image=False, cmap="Reds")
disp(viz, "Top 10% seams", s=15)
disp(expanded_image, title="Expanded by 10% horizontally", s=15)

w = image.shape[1]
expanded_image = expand_image(image, num_seams=int(w*0.25), seam_dir="vertical")
viz = visualize_best_seams(image, num_seams=int(w*0.25), seam_dir="vertical", on_image=False, cmap="Reds")
disp(viz, "Top 25% seams", s=15)
disp(expanded_image, title="Expanded by 25% horizontally", s=15)

w = image.shape[1]
expanded_image = expand_image(image, num_seams=int(w*0.50), seam_dir="vertical")
viz = visualize_best_seams(image, num_seams=int(w*0.50), seam_dir="vertical", on_image=False, cmap="Reds")
disp(viz, "Top 50% seams", s=15)
disp(expanded_image, title="Expanded by 50% horizontally", s=15)


In [ ]:
image = trump1
h = image.shape[0]
expanded_image = expand_image(image, num_seams=int(h*0.1), seam_dir="horizontal")
viz = visualize_best_seams(image, num_seams=int(h*0.1), seam_dir="horizontal", on_image=False, cmap="summer")
disp(viz, "Top 10% seams", s=15)
disp(expanded_image, title="Expanded by 10% vertically", s=15)

image = trump1
h = image.shape[0]
expanded_image = expand_image(image, num_seams=int(h*0.25), seam_dir="horizontal")
viz = visualize_best_seams(image, num_seams=int(h*0.25), seam_dir="horizontal", on_image=False, cmap="summer")
disp(viz, "Top 25% seams", s=15)
disp(expanded_image, title="Expanded by 25% vertically", s=15)

image = trump1
h = image.shape[0]
expanded_image = expand_image(image, num_seams=int(h*0.5), seam_dir="horizontal")
viz = visualize_best_seams(image, num_seams=int(h*0.5), seam_dir="horizontal", on_image=False, cmap="summer")
disp(viz, "Top 50% seams", s=15)
disp(expanded_image, title="Expanded by 50% vertically", s=15)

In [ ]:
image = landscape
w = image.shape[1]
h = image.shape[0]
expanded_image = expand_image(image, num_seams=int(w*0.3), seam_dir="vertical")
expanded_image = expand_image(expanded_image, num_seams=int(h*0.3), seam_dir="horizontal")
disp(expanded_image, title="Expanded by 30% vertically and horizontally", s=15)

### Trump Queen

In [ ]:
expanded, shrunk = remove_object(trump_queen, trump_queen_mask, save_animation=True, debug=True)

# cv2.imwrite("trump_queen_expanded.png", bgr(expanded))
# cv2.imwrite("trump_queen_shrunk.png", bgr(shrunk))
# cv2.imwrite("trump_queen.png", bgr(trump_queen))

# disp(trump_queen, title="Original", s=15)
# disp(shrunk, title="Object Removed", s=15)
# disp(expanded,title="Expanded", s=15)

### Trump Putin

In [ ]:
expanded, shrunk = remove_object(trump_putin, trump_putin_mask, save_animation=True, debug=True)

# cv2.imwrite("trump_putin_expanded2.png", bgr(expanded))
# cv2.imwrite("trump_putin_shrunk2.png", bgr(shrunk))
# cv2.imwrite("trump_putin2.png", bgr(trump_putin))

## Tests

In [ ]:
A = np.ones((10,10), dtype=np.uint8)
A[:5,:] = 0
A[0,:] = 1
A[:, 4:5] = 1
A[7:8, 3:6] = 0
A = 1-A
A = np.repeat(A[:, :, np.newaxis], 3, axis=2)*255
disp(A, s=6)

### Energy Map

In [ ]:
towers_energy = calculate_energy_map(towers)
disp(towers_energy, title="Energy Map")

In [ ]:
A_energy = A[:,:,0].astype(np.float64)
disp(A_energy, title="Energy Map")

### Find Seam

In [ ]:
seam, energy = find_best_seam(towers_energy)
drawn_towers = draw_seam(towers, seam, color=[255,0,0])
disp(drawn_towers, title="Seam")

In [ ]:
seam, energy = find_best_seam(A_energy, debug=True)
drawn_A = draw_seam(A, seam, color=[255,0,0])
disp(drawn_A, title="Seam")


### Shrink

In [ ]:
res = shrink_image(towers, 10, save_animation=False)
disp(res, title="Shrinked Towers")

### Mask 

In [ ]:
mask = np.zeros_like(towers, dtype=np.uint8)[:,:,0]
mask[100:200,100:200] = 255
mask[300:400,300:400] = 255
disp(mask)

In [ ]:
energy_map_mask = calculate_energy_map(towers, mask=mask)
seam, _= find_best_seam(energy_map_mask)
drawn_towers_mask = draw_seam(towers, seam, color=[255,0,0])
disp(cv2.addWeighted(drawn_towers_mask, 0.8, np.repeat(mask[:,:,np.newaxis],3,axis=2), 0.2, 0), title="Seam with Mask")

### insert seam

#### Grayscale seam insertion

In [ ]:
seam, energy = find_best_seam(A_energy, debug=False)
drawn_A_2 = draw_seam(A, seam, color=[255,0,0])
disp(drawn_A_2, title="Seam")

In [ ]:
A_inserted = insert_seam(A, seam)
disp(A_inserted, title="Inserted Seam")
print("A.shape", A.shape)
print("A_inserted.shape", A_inserted.shape)

#### Colored seam insertion

In [ ]:
seam, energy = find_best_seam(towers_energy, debug=False)
drawn_towers_2 = draw_seam(towers, seam, color=[255,0,0])
disp(drawn_towers_2, title="Seam")

In [ ]:
towers_inserted = insert_seam(towers, seam)
disp(towers_inserted, title="Inserted Seam")
print("towers.shape", towers.shape)
print("towers_inserted.shape", towers_inserted.shape)

### Naiive image expansion

In [ ]:
towers_expanded = expand_image_naive(towers, 400, save_animation=False)
disp(towers_expanded, title="Expanded Towers Naiive", s=11)
print("towers.shape", towers.shape)
print("towers_expanded.shape", towers_expanded.shape)

### Good image expansion

In [ ]:
towers_expanded2 = expand_image(towers, 400, save_animation=True)
disp(towers_expanded2, title="Expanded Towers Good", s=15)
print("towers.shape", towers.shape)
print("towers_expanded.shape", towers_expanded2.shape)

### Visualize seams

In [ ]:
tower_seams_drawn = visualize_best_seams(towers, num_seams=200, seam_dir="vertical", cmap="Reds", on_image=False)
disp(tower_seams_drawn, title="Best Seams", s=15)

In [ ]:
energy_map = calculate_energy_map(towers).astype(np.uint8)
        
# Convert energy map from 1 channel to 3 channels
energy_map_rgb = np.repeat(energy_map[:, :, np.newaxis], 3, axis=2)

disp(energy_map_rgb.astype(np.uint8), title="Energy Map")
print(energy_map_rgb.min(), energy_map_rgb.max())

### Remove object

#### example towers

In [ ]:
mask = np.zeros_like(towers, dtype=np.uint8)[:,:,0]
mask[10:350,330:600] = 255
# mask[300:400,300:400] = 255
disp(mask)

In [ ]:
tower_seams_drawn2 = visualize_best_seams(towers, num_seams=100, mask=mask, seam_dir="vertical", cmap="Reds", on_image=False)
disp(tower_seams_drawn2, title="Best Seams", s=15)

In [ ]:
mask_cp = mask.copy()
reduced = towers.copy()
for i in range(350):
    energy_test = calculate_energy_map(reduced, mask=mask_cp)
    # disp(energy_test, title="Energy Map")
    seam, energy = find_best_seam(energy_test)
    # disp(draw_seam(res, seam, color=[255,0,0]))
    reduced, mask_cp = remove_seam(reduced, seam, mask=mask_cp)
disp(reduced)

In [ ]:
mask_cp = mask.copy()
reduced = towers.copy()
i = 0
while (mask_cp>127).sum() > 0:
    i += 1
    energy_test = calculate_energy_map(reduced, mask=mask_cp)
    # disp(energy_test, title="Energy Map")
    seam, energy = find_best_seam(energy_test)
    # disp(draw_seam(res, seam, color=[255,0,0]))
    reduced, mask_cp = remove_seam(reduced, seam, mask=mask_cp)
disp(reduced)
print(f"Removed {i} seams to remove the mask entirely")

In [ ]:
towersless, mask_new = shrink_image(towers, 350, mask=mask, save_animation=True)
disp(towersless, title="Removed Towers", s=15)

#### example queen

In [ ]:
trumpqueen_seams_drawn = visualize_best_seams(trump_queen, num_seams=100, mask=trump_queen_mask, seam_dir="vertical", cmap="Reds", on_image=False)
disp(trumpqueen_seams_drawn, title="Best Seams", s=15)

In [ ]:
mask_cp = trump_queen_mask.copy()
reduced = trump_queen.copy()
i = 0
while (mask_cp>127).sum() > 0:
    i += 1
    energy_test = calculate_energy_map(reduced, mask=mask_cp)
    # disp(energy_test, title="Energy Map")
    seam, energy = find_best_seam(energy_test)
    # disp(draw_seam(res, seam, color=[255,0,0]))
    reduced, mask_cp = remove_seam(reduced, seam, mask=mask_cp)
disp(reduced)
print(f"Removed {i} seams to remove the queen entirely")

In [ ]:
queenless, mask_queen_new = shrink_image(trump_queen, 170, mask=trump_queen_mask, save_animation=True)
disp(queenless, title="Removed Queen", s=15)

In [ ]:
queen_expanded = expand_image(trump_queen, 170, save_animation=False)
disp(queen_expanded, title="Expanded Towers Good", s=15)
print("trump_queen.shape", trump_queen.shape)
print("queen_expanded.shape", queen_expanded.shape)

In [ ]:
queen_expanded = expand_image(queenless, 170, save_animation=False)
disp(queen_expanded, title="Expanded Towers Good", s=15)
disp(trump_queen, title="Expanded Queen", s=15)
print("queenless.shape", trump_queen.shape)
print("queen_expanded.shape", queen_expanded.shape)